# GADM 미국 county 정보 city 추가 ver. geojson to geom

필수 라이브러리 설치 및 필요한 function 정의

In [17]:
!pip install psycopg

In [18]:
import datetime
import requests
import json
import datetime

def send_api(url, method, jsonBody={}):
    headers = {'Content-Type': 'application/json', 'charset': 'UTF-8', 'Accept': '*/*'}
    try:
        if method == 'GET':
            response = requests.get(url, headers=headers)
        elif method == 'POST':
            response = requests.post(url, headers=headers, data=json.dumps(jsonBody, ensure_ascii=False, indent="\t"))
        print("response status %r: %s" %(response.status_code, url)) 
        return response.text
    except Exception as ex:
        print(ex)

# return type(dictionary obj)
def searchOpenMapAPI(word):
    queryString = "https://nominatim.openstreetmap.org/search.php?q=%s&format=jsonv2" %(word)
    res = json.loads(send_api(queryString, "GET"))
    return res

# param osmid=지역ID
# hierarchy= hierarchy 구조 노출여부(1:노출, 0:비노출)
# polygon_geojson= 폴리곤 좌표 노출여부(1:노출, 0:비노출)
# return type(dictionary obj)
def detailsOpenMapAPIByOsmid(osmid, hierarchy=0, polygon_geojson=0):
    queryString = "https://nominatim.openstreetmap.org/details.php?osmtype=R&osmid=%s&class=boundary&addressdetails=1&hierarchy=%s&group_hierarchy=1&format=json&polygon_geojson=%s" %(osmid, hierarchy, polygon_geojson)
    res = json.loads(send_api(queryString, "GET"))
    return res
    
def reversOpenMapAPI(lat, long, zoom):
    queryString = "https://nominatim.openstreetmap.org/reverse.php?lat=%s&lon=%s&zoom=%s&format=geojson&accept-language=en&extratags=1" %(lat, long, zoom)
    location = json.loads(send_api(queryString, "GET"))
    print(location,"\n")
    
    extratags = location["features"][0]["properties"]["extratags"]
    print("%s zooom extratags=%s" %(zoom, extratags))
    cityCode = extratags.get("ISO3166-2", None)
    print("%s zoom 도시코드: %s \n" %(zoom, cityCode))
    
    return cityCode




## gadm36_usa_2 테이블에서 각 county 정보 가져오자

- 대상 테이블: gadm36_usa_2


In [19]:
import psycopg
db = psycopg.connect(host='postgresql서버IP', dbname='template1', user='postgres', password='postgres1234', port=5432)
print(db)

# 검색 쿼리 return rows(type list)
def select(query):
    result=None
    try:
        with db.cursor() as cur:
            cur.execute(query, None)
            result= cur.fetchall()
    except (Exception, psycopg.DatabaseError) as error:
        print(error)
        
    return result

def excuteAndCommit(query):
    try:
        with db.cursor() as cur:
            cur.execute(query, None)
            db.commit()
    except  (Exception, psycopg.DatabaseError) as error:
        print(error)
        print(query)
        db.rollback()


gadm_query = "select name_1, name_2 from public.gadm36_usa_2 gu where name_1='Illinois' and name_2='Cook'"
rows = select(gadm_query)
print(rows)

# county addr
name_1 = rows[0][0]
name_2 = rows[0][1]
res = searchOpenMapAPI("%s %s" %(name_1, name_2))
if res[0]["type"]=='administrative':
    res_details_county = detailsOpenMapAPIByOsmid(res[0]["osm_id"], 1, 0)
    
    for city in res_details_county["hierarchy"]["administrative"]:
        res_details_city = detailsOpenMapAPIByOsmid(city["osm_id"], 0, 1)
        print(res_details_city["localname"])
        
        #print(res_details_city["geometry"]["coordinates"])
        geometry_type = res_details_city["geometry"]["type"]
        print(geometry_type)
        
        if geometry_type=='Polygon':
            sql_geom = "ST_Multi(ST_GeomFromGeoJSON('"+json.dumps(res_details_city["geometry"])+"'))"
        else:
            sql_geom = "ST_GeomFromGeoJSON('"+json.dumps(res_details_city["geometry"])+"')"
        
        insert_query="INSERT INTO public.gadm36_usa_3_kimjy (name_3, geom) VALUES( '%s', %s)" %(res_details_city["localname"], sql_geom)
        # print("query text length = " + str(len(insert_query)))
        # print(insert_query)
        # excuteAndCommit(insert_query)

db.close()
    

<psycopg.Connection [IDLE] (host=database-1.cbj3ip1sfdrz.ap-northeast-2.rds.amazonaws.com user=postgres database=template1) at 0x7f64b27f33a0>
[('Illinois', 'Cook')]
response status 200: https://nominatim.openstreetmap.org/search.php?q=Illinois Cook&format=jsonv2
response status 200: https://nominatim.openstreetmap.org/details.php?osmtype=R&osmid=122576&class=boundary&addressdetails=1&hierarchy=1&group_hierarchy=1&format=json&polygon_geojson=0
response status 200: https://nominatim.openstreetmap.org/details.php?osmtype=R&osmid=11584618&class=boundary&addressdetails=1&hierarchy=0&group_hierarchy=1&format=json&polygon_geojson=1
Barrington Township
Polygon
response status 200: https://nominatim.openstreetmap.org/details.php?osmtype=R&osmid=11587221&class=boundary&addressdetails=1&hierarchy=0&group_hierarchy=1&format=json&polygon_geojson=1
Bloom Township
Polygon
response status 200: https://nominatim.openstreetmap.org/details.php?osmtype=R&osmid=11587225&class=boundary&addressdetails=1&hie